In [12]:
import os
import pathlib
import json
import io
import tensorflow as tf
from random import shuffle
from shutil import rmtree
from object_detection.utils import label_map_util

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element

import pandas as pd

In [ ]:
PROJECT_CONFIG_PATH = 'workspace/kangaroos/training-config.json'

In [ ]:
def load_config():
    with open(PROJECT_CONFIG_PATH, 'r') as f:
        return json.load(f)
        

In [ ]:
PROJECT_CONFIG = load_config()

In [ ]:
def create_name_index():
    categories_map = label_map_util.create_categories_from_labelmap((PROJECT_CONFIG['PATH_TO_LABELS']))
    name_index = {}
    for cat in categories_map:
        name_index[cat['name']] = cat
    return name_index

In [ ]:
def xml_to_csv(annotations):
    xml_list = []
    for idx, xml_file in enumerate(annotations):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        size = root.find('size')
        
        width = int(size.find('width').text)
        height = int(size.find('height').text)
        
        filename = root.find('path').text.replace('\\','/')+'/'+root.find('filename').text
        count = 0
        for member in root.findall('object'):
            count = count + 1
            bndbox = member.find('bndbox')
            value = (filename,
                    width,
                    height,
                    member.find('name').text,
                    int(bndbox.find('xmin').text),
                    int(bndbox.find('ymin').text),
                    int(bndbox.find('xmax').text),
                    int(bndbox.find('ymax').text)
                    )
            xml_list.append(value)
        
        print('Processed {}. {} rows'.format(xml_file, count))
        
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax']
    return pd.DataFrame(xml_list, columns=column_name)

In [ ]:
def create_tf_record(group,category_map):
    with tf.compat.v1.gfile.GFile(group.filename, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(category_map[row['class']]['id'])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [13]:
def perform_partition(argv):    
    print(argv)
    split_percentage = 0.9
    annotation_names = [os.path.join(PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'],f) for f in os.listdir(PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'])]
    shuffle(annotation_names)
    num_annotations = len(annotation_names)
    num_train = int(num_annotations*split_percentage)
    train_annotations, test_annotations = annotation_names[0:num_train],annotation_names[num_train:]

    if os.path.exists(PROJECT_CONFIG['PASCAL_VOC_TRAIN_ANNOTATIONS_PATH']):
        rmtree(PROJECT_CONFIG['PASCAL_VOC_TRAIN_ANNOTATIONS_PATH'])

    if os.path.exists(PROJECT_CONFIG['PASCAL_VOC_VALIDATION_ANNOTATIONS_PATH']):
        rmtree(PROJECT_CONFIG['PASCAL_VOC_VALIDATION_ANNOTATIONS_PATH'])

    def generate_record_and_csv(source_base: str, target_base: str, annotations):
        if not os.path.exists(target_base):
            os.makedirs(target_base)

        xml_df = xml_to_csv(annotations)
        xml_df.to_csv(os.path.join(target_base,'annotations_csv.csv'), index=None)
        print('Finished outputting {}'.format(os.path.join(target_base,'annotations_csv.csv')))
        
        # Finished outputting the CSV record above, now converting it to a record
        with tf.compat.v1.python_io.TFRecordWriter(os.path.join(target_base,'tensorflow_record.record')) as writer:
            
            data = namedtuple('data', ['filename', 'object'])
            gb = xml_df.groupby('filename')
            grouped = [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]
            
            category_map = create_name_index()
            for group in grouped:
                tf_record = create_tf_record(group, category_map)
                writer.write(tf_record.SerializeToString())


    generate_record_and_csv(
        PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'], 
        PROJECT_CONFIG['PASCAL_VOC_TRAIN_ANNOTATIONS_PATH'], 
        train_annotations)

    generate_record_and_csv(
        PROJECT_CONFIG['PASCAL_VOC_ANNOTATIONS_PATH'], 
        PROJECT_CONFIG['PASCAL_VOC_VALIDATION_ANNOTATIONS_PATH'], 
        test_annotations)
    
    print('Vini','Vidi','Vici')

In [14]:
tf.compat.v1.app.run(perform_partition)

['c:\\users\\letlhogonolo segoe\\pycharmprojects\\models\\my-goodness-experiments\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\Letlhogonolo Segoe\\AppData\\Roaming\\jupyter\\runtime\\kernel-da71d12f-1420-454a-b49f-4c0a54056dd3.json']
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0029.xml. 2 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0082.xml. 3 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0155.xml. 1 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0036.xml. 1 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0084.xml. 1 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0066.xml. 3 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0083.xml. 1 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0099.xml. 2 rows
Processed D:/university/datasets/Kan

I0524 16:35:26.288796   452 label_map_util.py:120] Ignore item 0 since it falls outside of requested label range.


Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0095.xml. 2 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0151.xml. 2 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0085.xml. 2 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0002.xml. 2 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0146.xml. 2 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0061.xml. 6 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0119.xml. 3 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0101.xml. 1 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0118.xml. 8 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0016.xml. 1 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0120.xml. 2 rows
Processed D:/universi

I0524 16:35:27.410185   452 label_map_util.py:120] Ignore item 0 since it falls outside of requested label range.


Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0147.xml. 2 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0141.xml. 1 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0011.xml. 1 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0132.xml. 4 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0123.xml. 1 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0037.xml. 2 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0116.xml. 4 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0150.xml. 3 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0110.xml. 2 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0027.xml. 2 rows
Processed D:/university/datasets/Kangaroo/kangaroo/augmentedAnnotations\0017.xml. 2 rows
Processed D:/universi

SystemExit: 

c:\users\letlhogonolo segoe\pycharmprojects\models\my-goodness-experiments\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
